En este notebook spliteto el dataSet en 3 csv, uno para cada conjunto de datos

In [1]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import math
import numpy as np
import os

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [10]:
root = '.'
root = os.path.join(root,"..","data")
csv_path = os.path.join(".","..","csv")
csv_name = "masterDS"
os.listdir(csv_path)


['masterDS.csv', 'metaDataMaster.csv']

# Generando un csv para el training y para el validation set

In [12]:
original = pd.read_csv(os.path.join(csv_path,csv_name+".csv"))
df = original[original['sample_full'] == 'sample']
df.set_index('artist', inplace= True) #Decido el indice sobre el cual se va a iterar
df.head()

,MFI,album,song,original_path,original_file,16k_path,16k_file,sample_full
artist,,,,,,,,
Ariana Grande,mujer,Brand New You (From 13),01. Brand New You (feat. Brynn Williams & Cait...,/audio/mujer/Ariana Grande/Brand New You (From...,/audio/mujer/Ariana Grande/Brand New You (From...,..\data\masterDS,..\data\masterDS\01. Brand New You (feat. Bryn...,sample
Ariana Grande,mujer,Brand New You (From 13),01. Brand New You (feat. Brynn Williams & Cait...,/audio/mujer/Ariana Grande/Brand New You (From...,/audio/mujer/Ariana Grande/Brand New You (From...,..\data\masterDS,..\data\masterDS\01. Brand New You (feat. Bryn...,sample
Ariana Grande,mujer,Brand New You (From 13),01. Brand New You (feat. Brynn Williams & Cait...,/audio/mujer/Ariana Grande/Brand New You (From...,/audio/mujer/Ariana Grande/Brand New You (From...,..\data\masterDS,..\data\masterDS\01. Brand New You (feat. Bryn...,sample
Ariana Grande,mujer,Brand New You (From 13),01. Brand New You (feat. Brynn Williams & Cait...,/audio/mujer/Ariana Grande/Brand New You (From...,/audio/mujer/Ariana Grande/Brand New You (From...,..\data\masterDS,..\data\masterDS\01. Brand New You (feat. Bryn...,sample
Ariana Grande,mujer,Brand New You (From 13),01. Brand New You (feat. Brynn Williams & Cait...,/audio/mujer/Ariana Grande/Brand New You (From...,/audio/mujer/Ariana Grande/Brand New You (From...,..\data\masterDS,..\data\masterDS\01. Brand New You (feat. Bryn...,sample


In [13]:
classes = list(np.unique(df.index))
classes

['Ariana Grande', 'Katy Perry', 'Metallica', 'Pink Floyd', 'Taylor Swift']

In [14]:
dfDic = {}
m = float('inf')
for c in classes:
    dfDic[c] = df.loc[c]

for c in classes:
    l = len(dfDic[c])
    print(f"{c} : {l}")
    if (m > l):
        m = l

Ariana Grande : 3911
Katy Perry : 5419
Metallica : 11834
Pink Floyd : 6945
Taylor Swift : 3872


In [15]:
#mezclo los datasets, incluso antes de separarlos, asi saco de todas las canciones
dfRand = {}
for c in classes:
    dfRand[c] = dfDic[c].sample(frac=1)

In [16]:
#desimo los datasets de mujeres e instrumentales para que todas las categorias tengan la misma cantidad
# de muestrs

dfDes = {}
for c in classes:
    dfDes[c] = dfRand[c][:m]

In [17]:
# separo un 75% de las muestras para el trining y el 25% para los valid y tes sets

lTrain = math.floor(m * 0.75)
lVal = math.floor((m-lTrain) * 0.9)
lTest = m - lTrain - lVal

dfTrain = {}
dfVal = {}
dfTest = {}
for c in classes:
    dfTrain[c] = dfDes[c][:lTrain]
    dfVal[c] = dfDes[c][lTrain:lTrain+lVal]
    dfTest[c] = dfDes[c][lTrain+lVal:]

In [18]:
#muestro cuantas muestras tiene cada set
len(dfTrain[classes[0]]), len(dfVal[classes[0]]), len(dfTest[classes[0]])

(2904, 871, 97)

In [19]:
os.path.join(csv_path,csv_name+"train.csv")

'.\\..\\csv\\masterDStrain.csv'

In [20]:
#concateno los sets
train_set = pd.concat(dfTrain.values())
train_set.reset_index(inplace=True)
train_set.to_csv(os.path.join(csv_path,csv_name+"_train.csv"), index = False)

val_set = pd.concat(dfVal.values())
val_set.reset_index(inplace=True)
val_set.to_csv(os.path.join(csv_path,csv_name+"_val.csv"), index = False)

test_set = pd.concat(dfTest.values())
test_set.reset_index(inplace=True)
test_set.to_csv(os.path.join(csv_path,csv_name+"_test.csv"), index = False)

full_songs = original[original['sample_full'] == 'full']
full_songs.to_csv(os.path.join(csv_path,csv_name+"_full.csv"))